<a href="https://colab.research.google.com/github/fGizemCaliskan/bioinformatics-alzheimer-drug-predicter/blob/main/Alzheimer_Drug_Predicter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Moleküler yapıları işlemek için RDKit kütüphanesini kuruyoruz
!pip install rdkit

In [ ]:
import pandas as pd
import io
import requests

# Alternatif çalışan linkleri sırayla deneyecek güvenli bir yöntem
urls = [
    'https://raw.githubusercontent.com/dataprofessor/data/master/acetylcholinesterase_04_bioactivity_data_3class_pIC50.csv',
    'https://raw.githubusercontent.com/dataprofessor/data/master/acetylcholinesterase_03_bioactivity_data_curated.csv'
]

df = None

for url in urls:
    try:
        response = requests.get(url)
        if response.status_code == 200:
            df = pd.read_csv(io.StringIO(response.text))
            print(f"Veri başarıyla çekildi! Kaynak: {url}")
            break
    except:
        continue

if df is not None:
    # Bazen sütun isimleri farklı olabilir, pIC50 sütununun olduğundan emin olalım
    if 'pIC50' in df.columns:
        df = df[['canonical_smiles', 'pIC50']]
        print(f"Toplam Veri Sayısı: {len(df)}")
        display(df.head()) # İlk 5 satırı göster
    else:
        print("Hata: pIC50 sütunu bulunamadı. Lütfen başka bir veri seti deneyelim.")
else:
    print("Hata: Hiçbir linkten veri çekilemedi.")

In [ ]:
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import RDLogger

# 1. RDKit'in gereksiz uyarılarını (Deprecation Warning) susturalım
RDLogger.DisableLog('rdApp.*')

# 2. Fonksiyon: SMILES kodunu sayısal parmak izine çevirir
def smiles_to_fingerprint(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        # GÜNCELLEME: Eski yöntem yerine yeni 'MorganGenerator' kullanıyoruz
        # Bu sayede o kırmızı yazılar çıkmayacak.
        fp_gen = AllChem.GetMorganGenerator(radius=2, fpSize=2048)
        fp = fp_gen.GetFingerprintAsNumPy(mol)
        return fp
    else:
        return None

# 3. Tüm veri setine uygulayalım
print("Moleküller sayısal verilere dönüştürülüyor (Lütfen bekleyin)...")
X_data = [smiles_to_fingerprint(s) for s in df['canonical_smiles']]

# Hatalı (boş) dönüşümleri temizleyelim
X = []
y = []
for i, fp in enumerate(X_data):
    if fp is not None:
        X.append(fp)
        y.append(df['pIC50'].iloc[i])

X = np.array(X)
y = np.array(y)

print("Dönüşüm başarıyla tamamlandı!")
print(f"Girdi (X) boyutu: {X.shape}")

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error

# Veriyi %80 Eğitim, %20 Test olarak ayır
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modeli oluştur (100 ağaçlı bir orman)
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Modeli eğit
print("Model eğitiliyor (biraz sabır)...")
model.fit(X_train, y_train)

print("Eğitim bitti! Model artık molekülleri tanıyor.")

In [ ]:
# Test setindeki moleküllerin skorlarını tahmin et
y_pred = model.predict(X_test)

# Başarı skorunu hesapla (R2 skoru 1'e ne kadar yakınsa o kadar iyi)
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print(f"Modelin Başarı Skoru (R2): {r2:.2f}")
print(f"Hata Oranı (MSE): {mse:.2f}")

# İlk 5 tahmini görelim
print("\nGerçek Değer vs Tahmin:")
for i in range(5):
    print(f"Gerçek: {y_test[i]:.2f} | Tahmin: {y_pred[i]:.2f}")

In [ ]:
# Aspirin'in SMILES kodu
aspirin_smiles = "CC(=O)OC1=CC=CC=C1C(=O)O"

# Modele soralım
fp = smiles_to_fingerprint(aspirin_smiles)
tahmin = model.predict([fp])[0]

print(f"Aspirin için Tahmin Edilen pIC50 Değeri: {tahmin:.2f}")

if tahmin < 5:
    print("Sonuç: Zayıf etki (İlaç adayı olamaz)")
elif tahmin < 7:
    print("Sonuç: Orta etki")
else:
    print("Sonuç: Güçlü etki (Potansiyel ilaç!)")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Grafik ayarları
plt.figure(figsize=(7, 7))
sns.set(style="whitegrid")

# Noktaları çizdir (Mavi noktalar)
sns.scatterplot(x=y_test, y=y_pred, alpha=0.7, color='blue', edgecolor='k')

# İdeal doğrusu (Kırmızı çizgi - Tam isabet çizgisi)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', linewidth=2, linestyle='--')

plt.title(f'Gerçek vs Tahmin Edilen pIC50 (R2: {r2:.2f})', fontsize=16)
plt.xlabel('Gerçek Deneysel Değerler', fontsize=12)
plt.ylabel('Yapay Zeka Tahminleri', fontsize=12)
plt.show()

In [ ]:
# Test edilecek moleküller listesi (İsim ve SMILES kodu)
test_molekulleri = [
    ("Donepezil (Gerçek Alzheimer İlacı)", "COc1cc2c(cc1OC)C(=O)C(CC2)Cc3ccccc3"),
    ("Tacrine (Eski bir Alzheimer İlacı)", "Nc1c2ccccc2nc3ccccc13"),
    ("Rivastigmine (Alzheimer İlacı)", "CCN(C)C(=O)Oc1cccc(c1)C(C)N(C)C"),
    ("Paracetamol (Ağrı Kesici)", "CC(=O)Nc1ccc(O)cc1"),
    ("Ibuprofen (Ağrı Kesici)", "CC(C)Cc1ccc(c(c1)C)C(C)C(=O)O"),
    ("Kafein (Kahve)", "Cn1cnc2c1c(=O)n(C)c(=O)n2C"),
    ("Nikotin (Tütün)", "CN1CCC[C@H]1c2cccnc2"),
    ("Curcumin (Zerdeçal - Doğal)", "COc1cc(ccc1O)/C=C/C(=O)CC(=O)/C=C/c2ccc(O)c(OC)c2")
]

print(f"{'MOLEKÜL ADI':<35} | {'TAHMİN (pIC50)':<15} | {'SONUÇ'}")
print("-" * 70)

for isim, smiles in test_molekulleri:
    # 1. Parmak izine çevir
    fp = smiles_to_fingerprint(smiles)

    if fp is not None:
        # 2. Tahmin yap
        # predict fonksiyonu bir liste bekler, o yüzden [fp] şeklinde veriyoruz
        skor = model.predict([fp])[0]

        # 3. Yorumla
        if skor > 6.0:
            durum = "GÜÇLÜ ETKİ (İlaç Adayı!)"
        elif skor > 5.0:
            durum = "Orta Etki"
        else:
            durum = "Zayıf/Etkisiz"

        print(f"{isim:<35} | {skor:.2f}            | {durum}")
    else:
        print(f"{isim:<35} | HATA             | SMILES okunamadı")